<a href="https://colab.research.google.com/github/kunchek77/cars-test-/blob/main/NN_1_Project_Part_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DOMAIN: Electronics and Telecommunication

• CONTEXT: A communications equipment manufacturing company has a product which is responsible for emitting
informative signals. Company wants to build a machine learning model which can help the company to predict the
equipment’s signal quality using various parameters.

• DATA DESCRIPTION: The data set contains information on various signal tests performed:

1. Parameters: Various measurable signal parameters.

2. Signal_Quality: Final signal strength or quality

• PROJECT OBJECTIVE: The need is to build a regressor which can use these parameters to determine the signal strength or
quality [as number].

Steps and tasks: [ Total Score: 10 points]

1. Import data.

2. Data analysis & visualisation

• Perform relevant and detailed statistical analysis on the data.

• Perform relevant and detailed uni, bi and multi variate analysis.

Hint: Use your best analytical approach. Even you can mix match columns to create new columns which can be used for better
analysis. Create your own features if required. Be highly experimental and analytical here to find relevant hidden patterns.

3. Design, train, tune and test a neural network regressor.

Hint: Use best approach to refine and tune the data or the model. Be highly experimental here.

4. Pickle the model for future use.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import tensorflow as tf
import keras as kr
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Sequential # Forward prop
from keras.layers import Dense, Activation, LeakyReLU,ReLU
from keras import optimizers
from sklearn.preprocessing import Normalizer

Setting the display layout for the coding environment

In [ ]:
%load_ext tensorboard

print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

print("Tensor Flow Version: ",tf.__version__)
print("Keras Version: ",kr.__version__)

Import Dataset 

In [ ]:
df=pd.read_csv("/content/Part- 1,2&3 - Signal (1).csv")  # read csv file with ";" as seprators
df.head()  # Display top 5 rows of the dataset

2. Data analysis & visualisation

In [ ]:
 df.shape

In [ ]:
df.dtypes

In [ ]:
#Data type of each attribute 
df.info()

In [ ]:
df.describe(include="all")

In [ ]:
#Checking the presence of missing values
df.isnull()

In [ ]:
df.isnull().sum()   # Any of the values in the dataframe is a missing value

Perform relevant and detailed uni, bi and multi variate analysis.

In [ ]:
df.hist(figsize=(20,20),color="orange",grid=False)


In [ ]:
df.describe().transpose()

In [ ]:
cor=df.corr()
cor

In [ ]:
# However we want to see correlation in graphical representation so below is function for that in HEATMAP 
plt.subplots(figsize=(10,8))
sns.heatmap(cor,annot=True)

1.Parameter 6 and Parameter 7 are highly correlated with each other and visce versa and they have almost 0 correlation with other Parameters 

2.Parameter 1 is positively correlated to Parameter 3 and Parameter 8 and negatively correlated to Parameter 2 and Parameter 9. 

3.Parameter 4 is has very low correlation with other Parameters.

In [ ]:
sns.pairplot(df.iloc[:,1:])

In [ ]:
# Checking the presence of outliers
l = len(df)
col = list(df.columns)
#col.remove('condition')
for i in np.arange(len(col)):
    sns.boxplot(x= df[col[i]], color='cyan')
    plt.show()
    print('Boxplot of ',col[i])
    #calculating the outiers in attribute 
    Q1 = df[col[i]].quantile(0.25)
    Q2 = df[col[i]].quantile(0.50)
    Q3 = df[col[i]].quantile(0.75) 
    IQR = Q3 - Q1
    L_W = (Q1 - 1.5 *IQR)
    U_W = (Q3 + 1.5 *IQR)    
    print('Q1 is : ',Q1)
    print('Q2 is : ',Q2)
    print('Q3 is : ',Q3)
    print('IQR is:',IQR)
    print('Lower Whisker, Upper Whisker : ',L_W,',',U_W)
    bools = (df[col[i]] < (Q1 - 1.5 *IQR)) |(df[col[i]] > (Q3 + 1.5 * IQR))
    print('Out of ',l,' rows in data, number of outliers are:',bools.sum())   #calculating the number of outliers

In [ ]:
df.columns

In [ ]:
# create a boxplot for all the continuous features
df.boxplot(column = ['Parameter 1', 'Parameter 2', 'Parameter 3', 'Parameter 4',
       'Parameter 5', 'Parameter 6', 'Parameter 7', 'Parameter 8',
       'Parameter 9', 'Parameter 10', 'Parameter 11', 'Signal_Strength'], figsize = (20,10)) 

we see outliers in almost very parameter however highest in 4,6,7

Design, train, tune and test a neural network regressor.

In [ ]:
# counting the number of classes in output
df['Signal_Strength'].value_counts()

In [ ]:
#import pickle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import regularizers, optimizers
from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model

In [ ]:
X = df.drop("Signal_Strength", axis=1)
y = df['Signal_Strength']             


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
yc = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
tf.keras.backend.clear_session()
# splitting data for test of categorial 
Xcv_train, Xc_test, ycv_train, yc_test = train_test_split(X, yc, test_size=.20, random_state=1)

In [ ]:
print("Shape of y_train:", ycv_train.shape)
print("One value of y_train:", ycv_train[0])

In [ ]:
# splitting data for  train and validation of categorial 
Xc_train, Xc_val, yc_train, yc_val = train_test_split(Xcv_train, ycv_train, test_size=.20, random_state=1)

In [ ]:
print("Shape of y_train:", yc_train.shape)
print("One value of y_train:", yc_train[0])

Build the Graph

In [ ]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

In [ ]:
#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(11,)))

In [ ]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(128,kernel_initializer='normal', activation='relu'))

In [ ]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(60,kernel_initializer='normal'))


In [ ]:
#Add 3rd hidden layer
model.add(tf.keras.layers.Dense(32,kernel_initializer='normal'))
model.add(LeakyReLU(alpha=0.1))

In [ ]:
#Add 4th hidden layer
model.add(tf.keras.layers.Dense(16,kernel_initializer='normal'))
model.add(LeakyReLU(alpha=0.1))


In [ ]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(9, kernel_initializer='normal',activation='softmax'))


In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error', metrics=['accuracy'])

Review model

In [ ]:
model.summary()

Train the model

In [ ]:
  EPOCH=300
  model_cal=model.fit(x=Xc_train, y=yc_train, batch_size=32, epochs= EPOCH, validation_data=(Xc_val, yc_val))

In [ ]:
loss_train = model_cal.history['loss']
loss_val = model_cal.history['val_loss']
epochs = range(1,EPOCH+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
Acc_train = model_cal.history['accuracy']
Acc_val = model_cal.history['val_accuracy']
epochs = range(1,EPOCH+1)
plt.plot(epochs, Acc_train, 'g', label='Training accuracy')
plt.plot(epochs, Acc_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

Pickle the model for future use.

In [ ]:
#import pickle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import regularizers, optimizers
from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model

In [ ]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import load_model

In [ ]:
# load the model
model_cl = model.load_weights('/content/model.h5')
# serialize weights to HDF5
model.save_weights("/content/model.h5")
print("Saved model to disk")

In [ ]:
score = model.evaluate(Xc_train, yc_train, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
# score of test data
score_t = model.evaluate(Xc_test, yc_test, verbose=0)
print( score_t)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(y_pred[0])
print(y_pred[1])
print(y_pred[2])
print(y_pred[3])
print(y_pred[4])